In [2]:
import pandas as pd
import numpy as np
import serial
from pickle import load
import matplotlib.pyplot as plt
from plotly.offline import iplot 
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline()

from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
import tensorflow as tf
from keras.models import load_model
model1 = load_model('gru_model.h5')
scaler = load(open('scaler_ver2.pkl', 'rb'))

In [3]:
def scalerData(df):
    return scaler.transform(df)

In [4]:
def round3(x):
    return round(x,3)

In [5]:
def sortData(x,y,z):
    slist = x+y+z
    aa = np.array(slist).reshape(1,-1)
    return scalerData(aa)

In [6]:
def Predict(model):
    return round(model[0][0])

In [7]:
ser = serial.Serial() # 시리얼을 연결한다.
ser.port = 'COM3'
ser.baudrate = 115200 


ser.timeout = 1 #시리얼에 데이터를 불러올 때 지정하는 딜레이

# 시리얼을 열어준다.
ser.open()
a=[]
allG = []
allX = []
allY= []
aa=[]
Ecount = 0
n_count = 0
# 반복해서 데이터를 출력하기 위해 while 을 만들어주었다.
while ser.readable():
    try:
        data = ser.readline() 
        if len(data) > 23 and len(data)<30:
            data = str(data,'utf-8')
            data1 = list(map(round3,map(float,data.split(','))))
            Ecount = 0
            allG.append(data1[0])
            allX.append(data1[1])
            allY.append(data1[2])
            a.append(0)
            if len(allG) > 30:
                if Predict(model1.predict(sortData(allG[-30:], allX[-30:], allY[-30:]).reshape(-1,90,1))) == 1:
                    n_count +=1
#                     print(n_count)
                    if n_count == 1:
                        del a[0]
                        a.append(1)
                        print("낙상")
#                         break
#                 else:
#                     n_count = 0
#                     print(n_count)
        else:
            Ecount = Ecount+1
 
    except:
        Ecount = Ecount+1
        print("Error")
        
    if (Ecount == 5):
        print("End")
        break
ser.close() 
angle = {"AllG": allG,"AllX": allX ,"AllY": allY , "line" :a}

df = pd.DataFrame(angle)
df1 = df.iloc[30:,0:3]
plt.rcParams["figure.figsize"] = (10,8)
fig = df1.iplot(asFigure=True)
for i in df[df.line==1].index:
    fig.add_trace(go.Scatter(x=[i,i], y=[min(df.AllX),max(df.AllG)],line_color='red',text = f'{i}'))
#     fig.add_scatter(x=[i]*100, y=np.linspace(min(df.AllX),max(df.AllG),100), name=f'{i}')
fig.show()

낙상
End
